# Metrics

We have used simple functions in the previous assignments, such as MSE in regression problems or accuracy in classification problems, to quantify the goodness-of-fit of the model. While in regression problems the functions are based on an error calculated in one way or another (mean error, mean square error, etc.), in classification problems other types of metrics can be derived depending on what the problem being solved is like. Many of these metrics, at least those that will be used in practice, are based on the calculation of the confusion matrix.

A confusion matrix is a square matrix, with as many rows and columns as classes, showing the distribution of patterns in classes, and the classification performed by the model. Usually the rows show how the model has performed the classification, and the columns show the actual classification values, although this may vary depending on the source consulted.

The simplest case corresponds to 2 classes, where one is considered "negative" and the other "positive". A two-class confusion matrix would be as follows:

This confusion matrix contains 4 values, which can be divided

- according to the output of the model: positive or negative.
- depending on whether the model is wrong or not: true or false. 

Thus, these 4 values are called true negatives (TN), false positives (FP), false negatives (FN) and true positives (TP). For example, false negatives would be the number of patterns that the system has classified as negative, and has been wrong because they were actually positive.

From this confusion matrix, different metrics can be calculated. Depending on the problem you are working on, it will be more interesting to follow one or the other. Some of the most most commonly used metrics are:

- **Accuracy**. Ratio of patterns in which the prediction is correct. Calculated as $$\frac{TN+TP}{TN+TP+FN+FP}$$
- **Error rate**. Ratio of patterns in which the prediction is wrong. Calculated as $$\frac{FP+FN}{TN+TP+FN+FP}$$
- **Sensitivity** or **recall**. Indicates the probability that a positive classification result is obtained for a positive case. It is calculated as $$\frac{TP}{FN+TP}$$
    - In a medical test, the test sensitivity represents the probability that a sick (positive) subject will have a positive test result.
- **Specificity**. Indicates the probability that a negative classification result is obatined for a negative case. It is calculated as $$\frac{TN}{FP+TN}$$
    - The specificity of a test represents the probability that a healthy (negative) subject will have a negative test result.
- **Precision** or **positive predictive value**. Ratio of positive patterns that have been correctly classified. Calculated as $$\frac{TP}{TP+FP}$$
- **Negative predictive value**. Ratio of positive patterns that have been correctly classified. Calculated as $$\frac{TN}{TN+FN}$$
- **F-score**, **F1-score** or **F-measurement**. It is defined as the harmonic mean of precision and recall.

It is worth clarifying that these metrics, as well as others seen in theory class (ROC curve, Kappa index) are used to assess already trained classifiers, not to perform the training process. To be trained, each model has its own function to quantify the error or goodness of fit, such as the cross-entropy function in the case of neural networks.

The accuracy is probably the most commonly used value, as it indicates the success rate of the classifier in a simple way. However, depending on the problem you are working with, it may not be the most appropriate metric. For example, in a mass population-based test for a disease where it is known that most people do not have the disease, a model that classifies everyone as negative (healthy) will have a very high accuracy, even though the model does not actually do anything.

For this reason, it is necessary to assess which metric or metrics are the most used for each kind of problem. In many problems where the different classes are of equal importance, the accuracy value may be enough. However, in other problems, it may be of more interest to evaluate the situations in which a positive response is or should be produced by the model, as it could indicate something critical, such as detecting a disease or raising some kind of alarm. For this reason, sensitivity and positive predictive value values are often taken into account in addition to accuracy. There is a more extensive discussion of this in the theory notes, but a possible informal guide might be the following:

- If you want to minimise the number of positives incorrectly classified as negative (e.g. maximise the number of correctly diagnosed sick subjects, or maximise the number of alarms given for risky situations), the indicated metric is sensitivity (recall).
- If one wishes to minimise the number of samples incorrectly classified as positives (false positives, e.g. sick subjects diagnosed as healthy, or situations where an alarm should not has been raised but it was), the indicated metric is the positive predictive value (precision).

Therefore, the most appropriate metric depends entirely on the specific problem, according to the relative importance of the classifier output and its behaviour. In this type of problem, the F-score is a metric that may be more useful than accuracy.

Another issue to be considered is the data imbalance. Accuracy is a metric that gives a "global" view, which can be misleading when the distribution in classes is unbalanced. In these cases, F-score is a better metric. Having unbalanced databases is very common, which provides an extra argument for using F-score rather than accuracy.

Finally, if you have more than two classes, it is possible to build a confusion matrix in a similar way by having one row and column per class. For example:


In these cases, it is no longer possible to speak of positive or negative patterns, since there are more than two classes, nor to take values for sensitivity or positive predictive value. However, this confusion matrix can offer very interesting information when it comes to understanding how the model works, assessing which are the classes between which the model finds it easiest and most difficult to separate.

### Question
If the pattern set has been divided into training and test subsets, which subset should be used to calculated the confusion matrix?

**The test subset to see how the model performs on real world.**

In this assignment, you are asked to:

1. Develop a function called `confusionMatrix` which takes two vectors of equal length (the number of patterns), the first one containing the outputs obtained by a model (`outputs`) and the second with the desired outputs (`targets`), both of type `AbstractArray{Bool,1}`. This function should return:
    - Accuracy
    - Error rate
    - Sensitivity
    - Specificity
    - Positive predicitive value
    - Negative predicitve value
    - F-score
    - Confusion matrix, as an object of type `Array{Int64,2}` with two rows and two columns
    
    As this function is being fed with boolean-valued vectors, it will be applicable to problems with two classes (positive and negative cases).

    It is necessary to consider some particular situations when calculated the required classification metrics.
    
        - If every pattern is a true negative, neither the sensitivity nor the positive predictive value can be calculated. In this case the system works correctly, so these two metrics will be 1.
        - Similarly, neither the specificity nor the negative predictive value can be obtained and if every pattern is a true positive, so both metrics have to be manually set to 1.
        - If neither of these two cases has occurred and there is still any metric which cannot be calculated, it will take the value of 0. For example, if the sensitivity could not be calculated, it means that there was no pattern with a positive desired output.
        - It both sensitivity and positive predictive values are equal to 0, the value of F-score cannot be obtained, and thus it will be 0.
        
    Do not use loops inside the developed function.


In [1]:
function confusionMatrix(outputs::AbstractArray{Bool,1}, targets::AbstractArray{Bool,1})
    
    @assert(length(outputs) == length(targets))
    
    tp = sum(outputs .& targets)
    tn = sum(.!outputs .& .!targets)
    fp = sum(outputs .& .!targets)
    fn = sum(.!outputs .& targets)

    nPatterns = length(targets)

    accuracy = (tp + tn) / nPatterns
    errorRate = 1 - accuracy

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    precision = tp / (tp + fp)
    negative_predictive_value = tn / (tn + fn)

    if isnan(sensitivity) && isnan(precision)
        sensitivity = 1.
        precision = 1.
    elseif isnan(specificity) && isnan(negative_predictive_value)
        specificity = 1.
        negative_predictive_value = 1.
    end

    sensitivity = isnan(sensitivity) ? 0. : sensitivity
    precision = isnan(precision) ? 0. : precision
    specificity = isnan(specificity) ? 0. : specificity
    negative_predictive_value = isnan(negative_predictive_value) ? 0. : negative_predictive_value

    confusion_matrix = [tp fp; fn tn]

    fScore = (precision == sensitivity == 0) ? 0 : 2 * (precision * sensitivity) / (precision + sensitivity)

    return (accuracy=accuracy,
            errorRate=errorRate,
            sensitivity=sensitivity,
            specificity=specificity,
            precision=precision,
            negative_predictive_value=negative_predictive_value,
            fScore=fScore,
            confusion_matrix=confusion_matrix)
end


confusionMatrix (generic function with 1 method)

In [2]:
outputs_true_negative = [false, false, false, false, false]
targets_true_negative = [false, false, false, false, false]

print(confusionMatrix(outputs_true_negative, targets_true_negative), "\n")

outputs_true_positive = [true, true, true, true, true]
targets_true_positive = [true, true, true, true, true]

print(confusionMatrix(outputs_true_positive, targets_true_positive), "\n")

outputs_no_positive = [false, false, false, false, false]
targets_no_positive = [false, false, false, false, true]

print(confusionMatrix(outputs_no_positive, targets_no_positive), "\n")

outputs_no_negative = [true, true, true, true, true]
targets_no_negative = [true, true, true, true, false]

print(confusionMatrix(outputs_no_negative, targets_no_negative), "\n")

outputs_zero_sensitivity_ppv = [false, false, false, false, true]
targets_zero_sensitivity_ppv = [true, true, true, true, false]

print(confusionMatrix(outputs_zero_sensitivity_ppv, targets_zero_sensitivity_ppv), "\n")


(accuracy = 1.0, errorRate = 0.0, sensitivity = 1.0, specificity = 1.0, precision = 1.0, negative_predictive_value = 1.0, fScore = 1.0, confusion_matrix = [0 0; 0 5])
(accuracy = 1.0, errorRate = 0.0, sensitivity = 1.0, specificity = 1.0, precision = 1.0, negative_predictive_value = 1.0, fScore = 1.0, confusion_matrix = [5 0; 0 0])
(accuracy = 0.8, errorRate = 0.19999999999999996, sensitivity = 0.0, specificity = 1.0, precision = 0.0, negative_predictive_value = 0.8, fScore = 0, confusion_matrix = [0 0; 1 4])
(accuracy = 0.8, errorRate = 0.19999999999999996, sensitivity = 1.0, specificity = 0.0, precision = 0.8, negative_predictive_value = 0.0, fScore = 0.888888888888889, confusion_matrix = [4 1; 0 0])
(accuracy = 0.0, errorRate = 1.0, sensitivity = 0.0, specificity = 0.0, precision = 0.0, negative_predictive_value = 0.0, fScore = 0, confusion_matrix = [0 1; 4 0])


2. Many models (e.g. artificial neural networks) do not return a categorial output for each pattern, but the probability that it is "positive". For this reason, it is requested to develop a function with the same name as the previous one, whose first parameter is not a vector of boolean values but a vector of real values (of type `AbstractArray{<:Real}`). It also receives an optional third parameter with a threshold, with a default value, which is used to apply the previous function and return, therefore, the same values.

In [3]:
function confusionMatrix(outputs::AbstractArray{<:Real,1},targets::AbstractArray{Bool,1}; threshold::Real=0.5)
    outputs = outputs .> threshold
    return confusionMatrix(outputs, targets)
end

confusionMatrix (generic function with 2 methods)

In [4]:
outputs_true_negative = [0.7, 0.2, 0.55, 0.6, 0.1]
targets_true_negative = [true, false, true, false, true]

print(confusionMatrix(outputs_true_negative, targets_true_negative), "\n")

(accuracy = 0.6, errorRate = 0.4, sensitivity = 0.6666666666666666, specificity = 0.5, precision = 0.6666666666666666, negative_predictive_value = 0.5, fScore = 0.6666666666666666, confusion_matrix = [2 1; 1 1])


3. Develop two functions with the same name, `printConfusionMatrix`, that receive the model outputs and the desired outputs, call the previous functions and display the results obtained, including the confusion matrix. One of these functions shall receive a vector of model classifications (`outputs`) of type `AbstractArray{Bool,1}`, while for the other one this parameter shall be a vector of real values (of type `AbstractArray{<:Real}`). These functions will make calls to the previous functions.

In [36]:
function printConfusionMatrix(outputs::AbstractArray{Bool,1}, targets::AbstractArray{Bool,1})
    results = confusionMatrix(outputs, targets)

    accuracy = results.accuracy
    errorRate = results.errorRate
    sensitivity = results.sensitivity
    specificity = results.specificity
    precision = results.precision
    negative_predictive_value = results.negative_predictive_value
    fScore = results.fScore
    confusion_matrix = results.confusion_matrix

    println("Confusion Matrix:")
    println("True Positives:   ", confusion_matrix[1, 1])
    println("False Negatives:  ", confusion_matrix[2, 1])
    println("True Negatives:   ", confusion_matrix[2, 2])
    println("False Positives:  ", confusion_matrix[1, 2])

    println("\t\t\t\tActual")
    println("\t\t\t+----------+----------+")
    println("\t\t\t| Positive | Negative |")
    println("\t\t+------- +----------+----------+")
    println("\t\t| Positive|    ", confusion_matrix[1,1], "    |    ", confusion_matrix[1, 2], "    |")
    println("Predicted\t +----------+----------+----------+")
    println("\t\t| Negative|    ", confusion_matrix[2, 1], "    |    ", confusion_matrix[2,2], "    |")
    println("\t\t+----------+----------+----------+")

    print("Accuracy: ", round(accuracy * 100, digits=2), "% ")
    print("Error Rate: ", round(errorRate * 100, digits=2), "% ")
    print("Sensitivity (True Positive Rate): ", round(sensitivity * 100, digits=2), "% ")
    print("Specificity (True Negative Rate): ", round(specificity * 100, digits=2), "% ")
    print("Precision (Positive Predictive Value): ", round(precision * 100, digits=2), "% ")
    print("Negative Predictive Value: ", round(negative_predictive_value * 100, digits=2), "% ")
    print("F1 Score: ", round(fScore, digits=2))

end

printConfusionMatrix (generic function with 2 methods)

In [37]:
outputs = [true, true, false, false, true, false, true]
targets = [true, false, false, false, true, true, true]

printConfusionMatrix(outputs, targets)

Confusion Matrix:
True Positives:   3
False Negatives:  1
True Negatives:   2
False Positives:  1
				Actual
			+----------+----------+
			| Positive | Negative |
		+------- +----------+----------+
		| Positive|    3    |    1    |
Predicted	 +----------+----------+----------+
		| Negative|    1    |    2    |
		+----------+----------+----------+
Accuracy: 71.43% Error Rate: 28.57% Sensitivity (True Positive Rate): 75.0% Specificity (True Negative Rate): 66.67% Precision (Positive Predictive Value): 75.0% Negative Predictive Value: 66.67% F1 Score: 0.75

In [38]:
function printConfusionMatrix(outputs::AbstractArray{<:Real,1},targets::AbstractArray{Bool,1}; threshold::Real=0.5)
    outputs = outputs .> threshold
    return printConfusionMatrix(outputs, targets)
end

printConfusionMatrix (generic function with 2 methods)

In [39]:
outputs = [0.7, 0.8, 0.4, 0.3, 0.6, 0.1, 0.8]
targets = [true, false, false, false, true, true, true]

printConfusionMatrix(outputs, targets)

Confusion Matrix:
True Positives:   3
False Negatives:  1
True Negatives:   2
False Positives:  1
				Actual
			+----------+----------+
			| Positive | Negative |
		+------- +----------+----------+
		| Positive|    3    |    1    |
Predicted	 +----------+----------+----------+
		| Negative|    1    |    2    |
		+----------+----------+----------+
Accuracy: 71.43% Error Rate: 28.57% Sensitivity (True Positive Rate): 75.0% Specificity (True Negative Rate): 66.67% Precision (Positive Predictive Value): 75.0% Negative Predictive Value: 66.67% F1 Score: 0.75